In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import catboost

In [38]:
df = pd.read_csv('train.csv')
nulevie = np.array(df.notnull().sum())

for i in range(len(nulevie)):
    if nulevie[i] > 3100000:  
        df = df[df[df.columns[i]].isnull() != True]
df

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,3116940,e,9.29,f,NaN,n,t,NaN,NaN,w,...,b,NaN,w,u,w,t,g,NaN,d,u
3116941,3116941,e,10.88,s,NaN,w,t,d,c,p,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
3116942,3116942,p,7.82,x,e,e,f,a,NaN,w,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
3116943,3116943,e,9.45,p,i,n,t,e,NaN,p,...,NaN,y,w,NaN,NaN,t,p,NaN,d,u


In [39]:
clas = df['class']
clas = clas.replace('p',1).replace('e',0)
df = df.drop(['class'], axis = 1)

In [79]:
df_test = pd.read_csv('test.csv')

In [80]:
def dropp(df):
    df = df.drop(['veil-type','id','veil-color'],axis = 1)
    return df
df = pd.concat([df,df_test])
df = dropp(df)

In [81]:
cvv = df.columns[df.dtypes == object]
for i in cvv:
    sp = df[i].unique()
    for j in sp:
        if np.sum([df[i] == j]) < 300:
            df[i] = df[i].replace(j,np.nan)
    print(1)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [83]:
df['cap-surface'].unique()

array(['s', 'h', 'y', 'l', 't', 'e', 'g', 'd', 'i', 'w', 'k', nan],
      dtype=object)

In [72]:
df['cap-surface'].unique()


array(['s', 'h', 'y', 'l', 't', 'e', 'g', 'd', 'i', 'w', 'k'],
      dtype=object)

In [70]:
np.sum([df['cap-shape'] == 'x'])

1436467

In [84]:
df

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,has-ring,ring-type,spore-print-color,habitat,season
0,8.80,f,s,u,f,a,c,w,4.51,15.39,b,s,w,f,f,k,d,a
1,4.51,x,h,o,f,a,c,n,4.79,6.48,b,y,o,t,z,k,d,w
2,6.94,f,s,b,f,x,c,w,6.85,9.93,b,s,n,f,f,k,l,w
3,3.88,f,y,g,f,s,c,g,4.16,6.53,b,s,w,f,f,k,d,u
4,5.85,x,l,w,f,d,c,w,3.37,8.36,b,s,w,f,f,k,g,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077959,0.88,x,g,w,f,a,d,w,2.67,1.35,NaN,NaN,e,f,f,NaN,d,u
2077960,3.12,x,s,w,f,d,c,w,2.69,7.38,NaN,NaN,w,f,f,NaN,g,a
2077961,5.73,x,e,e,f,a,NaN,w,6.16,9.74,NaN,NaN,y,t,z,NaN,d,a
2077962,5.03,b,g,n,f,a,d,g,6.00,3.46,NaN,s,g,f,f,NaN,d,a


In [85]:
def prop(df):
    for j, i in enumerate(df.columns):
        if df.dtypes[j] != object:
            df[i] = df[i].fillna(df[i].mean(),inplace=False)
        else:
            df[i] = df[i].fillna(df[i].mode()[0],inplace=False)
    return df

In [86]:
df = prop(df)

C:\Users\allyo\AppData\Local\Temp\ipykernel_1528\3229977531.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != object:
C:\Users\allyo\AppData\Local\Temp\ipykernel_1528\3229977531.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != object:
C:\Users\allyo\AppData\Local\Temp\ipykernel_1528\3229977531.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != object:
C:\Users\allyo\AppD

In [87]:
df_test = df.iloc[3116717:]
df = df.iloc[:3116717]

In [88]:
df

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,has-ring,ring-type,spore-print-color,habitat,season
0,8.80,f,s,u,f,a,c,w,4.51,15.39,b,s,w,f,f,k,d,a
1,4.51,x,h,o,f,a,c,n,4.79,6.48,b,y,o,t,z,k,d,w
2,6.94,f,s,b,f,x,c,w,6.85,9.93,b,s,n,f,f,k,l,w
3,3.88,f,y,g,f,s,c,g,4.16,6.53,b,s,w,f,f,k,d,u
4,5.85,x,l,w,f,d,c,w,3.37,8.36,b,s,w,f,f,k,g,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,9.29,f,t,n,t,a,c,w,12.14,18.81,b,s,w,t,g,k,d,u
3116941,10.88,s,t,w,t,d,c,p,6.65,26.97,b,s,w,f,f,k,d,u
3116942,7.82,x,e,e,f,a,c,w,9.51,11.06,b,s,y,t,z,k,d,a
3116943,9.45,p,i,n,t,e,c,p,9.13,17.77,b,y,w,t,p,k,d,u


In [68]:
X_train, X_test, y_train, y_test = train_test_split(df, clas, test_size=0.2, random_state=42)
cat_feat = list(np.where(df.dtypes == object)[0])

In [75]:
model = catboost.CatBoostClassifier(iterations=430, learning_rate=0.4, max_depth=9)

model.fit(X_train,y_train,
         cat_features=cat_feat,
         eval_set=(X_test,y_test),
         verbose=False,
         plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [89]:
ppp = model.predict_proba(df_test)
predd = model.predict(df_test)
predd = np.where(predd == 0,'e','p')

In [136]:
ppp

array([[0.99713602, 0.00286398],
       [0.00325123, 0.99674877],
       [0.01564401, 0.98435599],
       ...,
       [0.00266172, 0.99733828],
       [0.99876858, 0.00123142],
       [0.9957061 , 0.0042939 ]])

In [90]:
predd

array(['e', 'p', 'p', ..., 'p', 'e', 'e'], dtype='<U1')

In [92]:
from sklearn.metrics import accuracy_score
ppredd1 = model.predict_proba(X_test)
ppredd = model.predict(X_test)
accuracy_score(ppredd,y_test)

0.9909151287250699

In [93]:
import xgboost

In [12]:
# df_test = prop(df_test)

C:\Users\allyo\AppData\Local\Temp\ipykernel_1384\1931310851.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != 'O':
C:\Users\allyo\AppData\Local\Temp\ipykernel_1384\1931310851.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != 'O':
C:\Users\allyo\AppData\Local\Temp\ipykernel_1384\1931310851.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != 'O':
C:\Users\allyo\AppData\Local

id
cap-diameter
cap-shape
cap-surface
cap-color
does-bruise-or-bleed


C:\Users\allyo\AppData\Local\Temp\ipykernel_1384\1931310851.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != 'O':
C:\Users\allyo\AppData\Local\Temp\ipykernel_1384\1931310851.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != 'O':
C:\Users\allyo\AppData\Local\Temp\ipykernel_1384\1931310851.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != 'O':


gill-attachment
gill-spacing
gill-color


C:\Users\allyo\AppData\Local\Temp\ipykernel_1384\1931310851.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != 'O':
C:\Users\allyo\AppData\Local\Temp\ipykernel_1384\1931310851.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != 'O':
C:\Users\allyo\AppData\Local\Temp\ipykernel_1384\1931310851.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != 'O':


...

veil-type
veil-color
has-ring


C:\Users\allyo\AppData\Local\Temp\ipykernel_1384\1931310851.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != 'O':
C:\Users\allyo\AppData\Local\Temp\ipykernel_1384\1931310851.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != 'O':
C:\Users\allyo\AppData\Local\Temp\ipykernel_1384\1931310851.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != 'O':


ring-type
spore-print-color
habitat


C:\Users\allyo\AppData\Local\Temp\ipykernel_1384\1931310851.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != 'O':
C:\Users\allyo\AppData\Local\Temp\ipykernel_1384\1931310851.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != 'O':
C:\Users\allyo\AppData\Local\Temp\ipykernel_1384\1931310851.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.dtypes[j] != 'O':


season


In [94]:
df = pd.concat([df,df_test])
df = pd.get_dummies(df)
df_test = df.iloc[3116717:]
df = df.iloc[:3116717]

In [96]:
X_train, X_test, y_train, y_test = train_test_split(df, clas, test_size=0.2, random_state=42)

In [121]:
params = {
        'n_estimators':300,
        'max_depth': 10,
        'learning_rate': 0.3,
        'grow_policy':'depthwise',
        'enable_categorical': True
}
model1 = xgboost.XGBClassifier(**params)
model1.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy='depthwise', importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [122]:
ppredd2 = model1.predict_proba(X_test)
ppredd3 = model1.predict(X_test)
accuracy_score(ppredd3,y_test)

0.9912680638620088

In [123]:
ppredd2

array([[9.9962312e-01, 3.7686864e-04],
       [1.4936328e-03, 9.9850637e-01],
       [1.0730803e-02, 9.8926920e-01],
       ...,
       [2.3612320e-01, 7.6387680e-01],
       [9.9296993e-01, 7.0300829e-03],
       [5.7214499e-04, 9.9942786e-01]], dtype=float32)

In [110]:
((ppredd1.T[0] + ppredd2.T[0])/2)

array([0.9984542 , 0.00208971, 0.00243021, ..., 0.10413424, 0.99362144,
       0.00410453])

In [106]:
iop = np.where(((ppredd1.T[0] + ppredd2.T[0])/2),1,0)
accuracy_score(iop,y_test)

0.5473109551066506

In [31]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,stem-width,14.446200
1,cap-surface,12.101995
2,stem-color,10.969422
3,gill-attachment,9.782605
4,gill-color,7.652369
5,stem-height,6.647715
6,stem-surface,6.391626
7,gill-spacing,6.336717
8,does-bruise-or-bleed,5.530227
9,cap-diameter,5.231744


In [133]:
preddxgb = model1.predict(df_test)
preddxgb = np.where(preddxgb == 0,'e','p')

In [156]:
sub = pd.read_csv('sample_submission.csv')
sub['class'] = p1
sub

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,e
...,...,...
2077959,5194904,p
2077960,5194905,p
2077961,5194906,p
2077962,5194907,e


In [157]:
sub.to_csv('pred10.csv',index=False)